# More Tips for Better Coding

## Introduction

This chapter covers more best practice tips for coding, although the advice is geard towards what is achievable and pragmatic rather than what is obtainable by a superhuman coding machine. Here, we'll cover more practical matters such as debugging code and logging.

## Debugging code

Computers are *very* literal, so literal that unless you're perfectly precise about what you want, they will end up doing something different. When that happens, one of the most difficult issues in programming is to understand *why* the code isn't doing what you expected. When the code doesn't do what we expect, it's called a bug.

Bugs could be fundamental issues with the code you're using (in fact, the term originated because a moth causing a problem in an early computer) and, if you find one of these, you should file an issue with the maintainers of the code. However, what's much more likely is that the instructions you gave aren't quite what is needed to produce the outcome that you want. And, in this case, you might need to *debug* the code: to find out which part of it isn't doing what you expect.

Even with a small code base, it can be tricky to track down where the bug is: but don't fear, there are tools on hand to help you find where the bug is.

### Print statements

The simplest, and I'm afraid to say the most common, way to debug code is to plonk `print` statements in the code. Let's take a common example in which we perform some simple array operations, here multiplying an array and then summing it with another array:

In [ ]:
import numpy as np


def array_operations(in_arr_one, in_arr_two):
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, '7', 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

Oh no! We've got a `UFuncTypeError` here, perhaps not the most illuminating error message we've ever seen. We'd like to know what's going wrong here. The `Traceback` did give us a hint about where the issue occurred though; it happens in the multiplication line of the function we wrote.

To debug the error with print statements, we might re-run the code like this:

In [ ]:
def array_operations(in_arr_one, in_arr_two):
    print(f'in_arr_one is {in_arr_one}')
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, '7', 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

What can we tell from the values of `in_arr_one` that are now being printed? Well, they seem to have quote marks around them and what that means is that they're strings, *not* floating point numbers or integers! Multiplying a string by 1.5 doesn't make sense here, so that's our error. If we did this, we might then trace the origin of that array back to find out where it was defined and see that instead of `np.array([3, 2, 5, 16, 7, 8, 9, 22])` being declared, we have `np.array([3, 2, 5, 16, '7', 8, 9, 22])` instead and `numpy` decides to cast the whole array as a string to ensure consistency.

Let's fix that problem by turning `'7'` into `7` and run it again:

In [ ]:
def array_operations(in_arr_one, in_arr_two):
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

Still not working! But we've moved on to a different error now. We can still use a print statement to debug this one, which seems to be related to the shapes of variables passed into the function:

In [ ]:
def array_operations(in_arr_one, in_arr_two):
    print(f'in_arr_one shape is {in_arr_one.shape}')
    out_arr = in_arr_one*1.5
    print(f'intermediate out_arr shape is {out_arr.shape}')
    print(f'in_arr_two shape is {in_arr_two.shape}')
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

The print statement now tells us the shapes of the arrays as we go through the function. We can see that in the line before the `return` statement the two arrays that are being combined using the `+` operator don't have the same shape, so we're effectively adding two vectors from two differently dimensioned vector spaces and, understandably, we are being called out on our nonsense. To fix this problem, we would have to ensure that the input arrays are the same shape (it looks like we may have just missed a value from `in_vals_two`).

`print` statements are great for a quick bit of debugging and you are likely to want to use them more frequently than any other debugging tool. However, for complex, nested code debugging, they aren't always very efficient and you will sometimes feel like you are playing battleships in continually refining where they should go until you have pinpointed the actual problem, so they're far from perfect. Fortunately, there are other tools in the debugging toolbox...


### Icecream and better print statements

Typing `print` statements with arguments that help you debug code can become tedious. There are better ways to work, which we'll come to, but we must also recognise that `print` is used widely in practice. So what if we had a function that was as easier to use as `print` but better geared toward debugging? Well, we do, and it's called [**icecream**], and it's available in most major languages, including Python, Dart, Rust, javascript, C++, PHP, Go, Ruby, and Java. 

Let's take an example from earlier in this chapter, where we used a `print` statement to display the contents of `in_arr_one` in advance of the line that caused an error being run. All we will do now is switch out `print(f'in_arr_one is {in_arr_one}')` for `ic(in_arr_one)`.

In [ ]:
from icecream import ic

def array_operations(in_arr_one, in_arr_two):
    # Old debug line using `print`
    # print(f'in_arr_one is {in_arr_one}')
    # new debug line:
    ic(in_arr_one)
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, '7', 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

array_operations(in_vals_one, in_vals_two)

What we get in terms of debugging output is `ic| in_arr_one: array(['3', '2', '5', '16', '7', '8', '9', '22'], dtype='<U21')`, which is quite similar to before apart from three important differences, all of which are advantages:

1. it is easier and quicker to write `ic(in_arr_one)` than `print(f'in_arr_one is {in_arr_one}')`

2. **icecream** automatically picks up the name of the variable, `in_arr_one`, and clearly displays its contents

3. **icecream** shows us that `in_arr_one` is of `type` array and that it has the `dtype` of `U`, which stands for Unicode (i.e. a string). `<U21` just means that all strings in the array are less than 21 characters long.

**icecream** has some other advantages relative to print statements too, for instance it can tell you about which lines were executed in which scripts if you call it without arguments:


In [ ]:
def foo():
    ic()
    print('first')
    
    if 10 < 20:
        ic()
        print('second')
    else:
        ic()
        print('Never executed')

foo()

And it can wrap assignments rather than living on its own lines:

In [ ]:
def half(i):
    return ic(i) / 2

a = 6
b = ic(half(a))

All in all, if you find yourself using `print` to debug, you might find a one-time import of **icecream** followed by use of `ic` instead both more convenient and more effective.

### Debugging with the IDE

In this section, we'll learn about how your Integrated Development Environment, or IDE, can aid you with debugging. While we'll talk through the use of Visual Studio Code, which is free, directly supports Python, R, and other languages, and is especially rich, many of the features will be present in other IDEs too and the ideas are somewhat general.

